## setup libraries

In [1]:
#library and code setup
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install -q pyspark
import pyspark, os
from pyspark import SparkConf, SparkContext
os.environ["PYSPARK_PYTHON"]="python3"
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64/"

!pip install gdelt

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,778 kB]
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 htt

In [2]:
#start spark local server
import sys, os
from operator import add
import time

os.environ["PYSPARK_PYTHON"]="python3"

import pyspark
from pyspark import SparkConf, SparkContext

#connects our python driver to a local Spark JVM running on the Google Colab server virtual machine
try:
  conf = SparkConf().setMaster("local[*]").set("spark.executor.memory", "1g")
  sc = SparkContext(conf = conf)
except ValueError:
  #it's ok if the server is already started
  pass

def dbg(x):
  """ A helper function to print debugging information on RDDs """
  if isinstance(x, pyspark.RDD):
    print([(t[0], list(t[1]) if 
            isinstance(t[1], pyspark.resultiterable.ResultIterable) else t[1])
           if isinstance(t, tuple) else t
           for t in x.take(100)])
  else:
    print(x)

## fetch gdelt data for a given date range

Note: this example only fetches the events table. The gdelt python API also can pull from the mentions and gkg tables. See https://linwoodc3.github.io/gdeltPyR/ for full API details



In [3]:
# these remove files from previous runs
!rm -rf articles
!rm *.csv

rm: cannot remove '*.csv': No such file or directory


In [4]:
from concurrent.futures import ProcessPoolExecutor
from datetime import date, timedelta
import pandas as pd
import gdelt
import os

# set up gdeltpyr for version 2
gd = gdelt.gdelt(version=2)

# multiprocess the query
e = ProcessPoolExecutor()
lst_data = []

# generic functions to pull and write data to disk based on date
def get_filename(x):
  date = x.strftime('%Y%m%d')
  return "{}_gdeltdata.csv".format(date)


def intofile(filename):
    try:
        if not os.path.exists(filename):
          date = filename.split("_")[0]
          d = gd.Search(date, table='gkg',coverage=False) #not updata at 15mins
          d.to_csv(filename,encoding='utf-8',index=False)
          return filename
    except:
        print("Error occurred")

# test gkg table
test_data = [get_filename(x) for x in pd.date_range('2021 May 1','2021 Nov 30')]
dbg(test_data)
#downloading
results = list(e.map(intofile,test_data))


['20210501_gdeltdata.csv', '20210502_gdeltdata.csv', '20210503_gdeltdata.csv', '20210504_gdeltdata.csv', '20210505_gdeltdata.csv', '20210506_gdeltdata.csv', '20210507_gdeltdata.csv', '20210508_gdeltdata.csv', '20210509_gdeltdata.csv', '20210510_gdeltdata.csv', '20210511_gdeltdata.csv', '20210512_gdeltdata.csv', '20210513_gdeltdata.csv', '20210514_gdeltdata.csv', '20210515_gdeltdata.csv', '20210516_gdeltdata.csv', '20210517_gdeltdata.csv', '20210518_gdeltdata.csv', '20210519_gdeltdata.csv', '20210520_gdeltdata.csv', '20210521_gdeltdata.csv', '20210522_gdeltdata.csv', '20210523_gdeltdata.csv', '20210524_gdeltdata.csv', '20210525_gdeltdata.csv', '20210526_gdeltdata.csv', '20210527_gdeltdata.csv', '20210528_gdeltdata.csv', '20210529_gdeltdata.csv', '20210530_gdeltdata.csv', '20210531_gdeltdata.csv', '20210601_gdeltdata.csv', '20210602_gdeltdata.csv', '20210603_gdeltdata.csv', '20210604_gdeltdata.csv', '20210605_gdeltdata.csv', '20210606_gdeltdata.csv', '20210607_gdeltdata.csv', '20210608_g

/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_ba

Error occurred


/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_ba

Error occurred


...

/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_ba

Error occurred


/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/parallel.py:103: UserWarning: GDELT did

Error occurred


/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_ba

In [5]:
#寻找有用的日期
result = []
for i in results:
  if i != None:
    result.append(i)
result

['20210501_gdeltdata.csv',
 '20210502_gdeltdata.csv',
 '20210503_gdeltdata.csv',
 '20210504_gdeltdata.csv',
 '20210506_gdeltdata.csv',
 '20210507_gdeltdata.csv',
 '20210508_gdeltdata.csv',
 '20210509_gdeltdata.csv',
 '20210510_gdeltdata.csv',
 '20210511_gdeltdata.csv',
 '20210512_gdeltdata.csv',
 '20210513_gdeltdata.csv',
 '20210514_gdeltdata.csv',
 '20210515_gdeltdata.csv',
 '20210516_gdeltdata.csv',
 '20210517_gdeltdata.csv',
 '20210518_gdeltdata.csv',
 '20210519_gdeltdata.csv',
 '20210520_gdeltdata.csv',
 '20210521_gdeltdata.csv',
 '20210522_gdeltdata.csv',
 '20210523_gdeltdata.csv',
 '20210524_gdeltdata.csv',
 '20210525_gdeltdata.csv',
 '20210527_gdeltdata.csv',
 '20210528_gdeltdata.csv',
 '20210529_gdeltdata.csv',
 '20210530_gdeltdata.csv',
 '20210531_gdeltdata.csv',
 '20210601_gdeltdata.csv',
 '20210602_gdeltdata.csv',
 '20210603_gdeltdata.csv',
 '20210604_gdeltdata.csv',
 '20210605_gdeltdata.csv',
 '20210607_gdeltdata.csv',
 '20210608_gdeltdata.csv',
 '20210609_gdeltdata.csv',
 

## read downloaded files into RDDs

In [6]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

test_data1 = sqlContext.read.option("header", "true").csv(result)
dbg(test_data)
test_data_rdd = test_data1.rdd
dbg(test_data_rdd)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


['20210501_gdeltdata.csv', '20210502_gdeltdata.csv', '20210503_gdeltdata.csv', '20210504_gdeltdata.csv', '20210505_gdeltdata.csv', '20210506_gdeltdata.csv', '20210507_gdeltdata.csv', '20210508_gdeltdata.csv', '20210509_gdeltdata.csv', '20210510_gdeltdata.csv', '20210511_gdeltdata.csv', '20210512_gdeltdata.csv', '20210513_gdeltdata.csv', '20210514_gdeltdata.csv', '20210515_gdeltdata.csv', '20210516_gdeltdata.csv', '20210517_gdeltdata.csv', '20210518_gdeltdata.csv', '20210519_gdeltdata.csv', '20210520_gdeltdata.csv', '20210521_gdeltdata.csv', '20210522_gdeltdata.csv', '20210523_gdeltdata.csv', '20210524_gdeltdata.csv', '20210525_gdeltdata.csv', '20210526_gdeltdata.csv', '20210527_gdeltdata.csv', '20210528_gdeltdata.csv', '20210529_gdeltdata.csv', '20210530_gdeltdata.csv', '20210531_gdeltdata.csv', '20210601_gdeltdata.csv', '20210602_gdeltdata.csv', '20210603_gdeltdata.csv', '20210604_gdeltdata.csv', '20210605_gdeltdata.csv', '20210606_gdeltdata.csv', '20210607_gdeltdata.csv', '20210608_g

In [7]:
test_data1.show()

+-----------------+--------------+--------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+
|      GKGRECORDID|          DATE|SourceCollectionIdentifier|    SourceCommonName|  DocumentIdentifier|              Counts|            V2Counts|              Themes|            V2Themes|           Locations|         V2Locations|             Persons|           V2Persons|       Organizations|     V2Organizations|              V2Tone|               Dates|                GCAM|        SharingImage|       RelatedImages|   SocialImageEmbeds|

In [8]:
#take country and theme as rdd 
countryName_And_Theme = test_data_rdd.map(lambda a: (a.Locations, a.Themes))
dbg(countryName_And_Theme)

[(None, 'CYBER_ATTACK;WB_678_DIGITAL_GOVERNMENT;WB_670_ICT_SECURITY;WB_133_INFORMATION_AND_COMMUNICATION_TECHNOLOGIES;TAX_FNCACT;TAX_FNCACT_CHIEF;TAX_MILITARY_TITLE;TAX_MILITARY_TITLE_OFFICER;TAX_FNCACT_OFFICER;LEADER;TAX_FNCACT_PRESIDENT;USPEC_POLITICS_GENERAL1;TAX_FNCACT_VICE_PRESIDENT;CRISISLEX_C07_SAFETY;TECH_AUTOMATION;TAX_FNCACT_CEO;TAX_FNCACT_LEADER;WB_652_ICT_APPLICATIONS;WB_666_APPLICATION_DEVELOPMENT;WB_658_ENTERPRISE_APPLICATIONS;WB_694_BROADCAST_AND_MEDIA;'), (None, 'WB_1979_NATURAL_RESOURCE_MANAGEMENT;WB_435_AGRICULTURE_AND_FOOD_SECURITY;WB_1986_MOUNTAINS;UNGP_FORESTS_RIVERS_OCEANS;'), ('1#Haiti#HA#HA#19#-72.416667#HA', 'EPU_POLICY;EPU_POLICY_POLITICAL;NATURAL_DISASTER;NATURAL_DISASTER_HURRICANE;CRISISLEX_CRISISLEXREC;TAX_ETHNICITY;TAX_ETHNICITY_HAITIAN;TAX_FNCACT;TAX_FNCACT_USHER;TAX_FNCACT_CEO;ECON_ENTREPRENEURSHIP;TAX_FNCACT_ENTREPRENEUR;UNGP_FORESTS_RIVERS_OCEANS;'), ('1#United States#US#US#39.828175#-98.5795#US;1#Israel#IS#IS#31.5#34.75#IS;2#Utah, United States#US#USU

In [9]:
#make country as a key
def find_country(string):
  """spilt the str and find the country"""
  if string == None:
    return str("Not found")
  lst = string.split('#')
  if len(lst) >= 3:
    return lst[2]
  else:
    return str("Not found")
take_country = countryName_And_Theme.map(lambda a: (find_country(a[0]), a[1])).filter(lambda a: a[0] != "Not found")
dbg(take_country)

[('HA', 'EPU_POLICY;EPU_POLICY_POLITICAL;NATURAL_DISASTER;NATURAL_DISASTER_HURRICANE;CRISISLEX_CRISISLEXREC;TAX_ETHNICITY;TAX_ETHNICITY_HAITIAN;TAX_FNCACT;TAX_FNCACT_USHER;TAX_FNCACT_CEO;ECON_ENTREPRENEURSHIP;TAX_FNCACT_ENTREPRENEUR;UNGP_FORESTS_RIVERS_OCEANS;'), ('US', 'PROTEST;STRIKE;GENERAL_HEALTH;TAX_DISEASE;TAX_DISEASE_DISEASE;WB_1406_DISEASES;WB_621_HEALTH_NUTRITION_AND_POPULATION;UNGP_HEALTHCARE;WB_2024_ANTI_CORRUPTION_AUTHORITIES;WB_696_PUBLIC_SECTOR_MANAGEMENT;WB_831_GOVERNANCE;WB_832_ANTI_CORRUPTION;WB_2026_PREVENTION;MANMADE_DISASTER_IMPLIED;PUBLIC_TRANSPORT;MEDICAL;SOC_POINTSOFINTEREST;SOC_POINTSOFINTEREST_HOSPITALS;AFFECT;POVERTY;CRISISLEX_C05_NEED_OF_SHELTERS;ACT_YIELD;CRISISLEX_C07_SAFETY;HEALTH_VACCINATION;WB_642_CHILD_HEALTH;WB_1459_IMMUNIZATIONS;WB_639_REPRODUCTIVE_MATERNAL_AND_CHILD_HEALTH;KILL;HEALTH_PANDEMIC;WB_635_PUBLIC_HEALTH;WB_2165_HEALTH_EMERGENCIES;WB_2166_HEALTH_EMERGENCY_PREPAREDNESS_AND_DISASTER_RESPONSE;WB_2167_PANDEMICS;TAX_ETHNICITY;TAX_ETHNICITY_AMERI

In [10]:
#make theme as a list
def to_list(string):
  if string == None:
    return str("No Theme")
  else:
    res = string.split(";")
    return res[:-1]
Country_And_Theme = take_country.map(lambda a: (a[0], to_list(a[1]))).filter(lambda a: a[1] != "No Theme")
dbg(Country_And_Theme)

[('HA', ['EPU_POLICY', 'EPU_POLICY_POLITICAL', 'NATURAL_DISASTER', 'NATURAL_DISASTER_HURRICANE', 'CRISISLEX_CRISISLEXREC', 'TAX_ETHNICITY', 'TAX_ETHNICITY_HAITIAN', 'TAX_FNCACT', 'TAX_FNCACT_USHER', 'TAX_FNCACT_CEO', 'ECON_ENTREPRENEURSHIP', 'TAX_FNCACT_ENTREPRENEUR', 'UNGP_FORESTS_RIVERS_OCEANS']), ('US', ['PROTEST', 'STRIKE', 'GENERAL_HEALTH', 'TAX_DISEASE', 'TAX_DISEASE_DISEASE', 'WB_1406_DISEASES', 'WB_621_HEALTH_NUTRITION_AND_POPULATION', 'UNGP_HEALTHCARE', 'WB_2024_ANTI_CORRUPTION_AUTHORITIES', 'WB_696_PUBLIC_SECTOR_MANAGEMENT', 'WB_831_GOVERNANCE', 'WB_832_ANTI_CORRUPTION', 'WB_2026_PREVENTION', 'MANMADE_DISASTER_IMPLIED', 'PUBLIC_TRANSPORT', 'MEDICAL', 'SOC_POINTSOFINTEREST', 'SOC_POINTSOFINTEREST_HOSPITALS', 'AFFECT', 'POVERTY', 'CRISISLEX_C05_NEED_OF_SHELTERS', 'ACT_YIELD', 'CRISISLEX_C07_SAFETY', 'HEALTH_VACCINATION', 'WB_642_CHILD_HEALTH', 'WB_1459_IMMUNIZATIONS', 'WB_639_REPRODUCTIVE_MATERNAL_AND_CHILD_HEALTH', 'KILL', 'HEALTH_PANDEMIC', 'WB_635_PUBLIC_HEALTH', 'WB_2165_HE

In [11]:
type(Country_And_Theme)

pyspark.rdd.PipelinedRDD

## start to looking at one country

Themes_of_CH:


*  [item1, item2......],
*  [item1, item2, item3......]
*  ....



In [12]:
Themes_of_CH = Country_And_Theme.filter(lambda a: a[0] == 'CH').map(lambda a: a[1])
dbg(Themes_of_CH)
test = Themes_of_CH.collect()

[['ECON_WORLDCURRENCIES', 'ECON_WORLDCURRENCIES_YUAN', 'ECON_WORLDCURRENCIES_DOLLARS', 'ECON_CENTRALBANK', 'WB_1235_CENTRAL_BANKS', 'WB_318_FINANCIAL_ARCHITECTURE_AND_BANKING', 'WB_1920_FINANCIAL_SECTOR_DEVELOPMENT', 'WB_1234_BANKING_INSTITUTIONS', 'USPEC_POLICY1', 'EPU_POLICY', 'EPU_POLICY_CENTRAL_BANK', 'EPU_CATS_MONETARY_POLICY', 'EPU_POLICY_PEOPLE_BANK_OF_CHINA', 'EPU_ECONOMY', 'EPU_ECONOMY_HISTORIC', 'EPU_POLICY_POLICY', 'WB_444_MONETARY_POLICY', 'WB_439_MACROECONOMIC_AND_STRUCTURAL_POLICIES', 'EPU_POLICY_MONETARY_POLICY'], ['EPU_ECONOMY_HISTORIC', 'TAX_ETHNICITY', 'TAX_ETHNICITY_AUSTRALIAN', 'TAX_FNCACT', 'TAX_FNCACT_MINISTER', 'TAX_FNCACT_FOREIGN_MINISTER', 'WB_2024_ANTI_CORRUPTION_AUTHORITIES', 'WB_696_PUBLIC_SECTOR_MANAGEMENT', 'WB_840_JUSTICE', 'WB_2025_INVESTIGATION', 'WB_831_GOVERNANCE', 'WB_832_ANTI_CORRUPTION', 'WB_1014_CRIMINAL_JUSTICE', 'TAX_DISEASE', 'TAX_DISEASE_CORONAVIRUS', 'GENERAL_HEALTH', 'HEALTH_PANDEMIC', 'WB_635_PUBLIC_HEALTH', 'WB_2165_HEALTH_EMERGENCIES', 'W

individual:


*   (item1, 10)
*   (item2,1)
* ....



In [13]:
#find frequence set and broadcast
individual = Themes_of_CH.flatMap(lambda a:[(i,1) for i in a]).reduceByKey(lambda a,b:a+b)
individual1 = individual.filter(lambda a: a[1] >= 50).collectAsMap()   #change the frequence here !!!!!!!!

individual_broad = sc.broadcast(individual1)
dbg(individual)

[('WB_1235_CENTRAL_BANKS', 178), ('TAX_FNCACT_CEO', 269), ('WB_1045_TREASURY', 129), ('ECON_OILPRICE', 100), ('TAX_ETHNICITY_VENEZUELAN', 6), ('WB_724_HUMAN_RESOURCES_FOR_PUBLIC_SECTOR', 340), ('LEGISLATION', 705), ('WB_2482_RECONCILIATION', 53), ('TAX_WEAPONS_TEAR_GAS', 4), ('WB_1427_NON_COMMUNICABLE_DISEASE_AND_INJURY', 191), ('BULLYING', 30), ('USPEC_UNCERTAINTY1', 529), ('SOC_EMERGINGTECH', 98), ('WB_512_GAS_FIRED_POWER', 5), ('TAX_ETHNICITY_HUNGARIAN', 7), ('WB_1841_SHORT_LIVED_CLIMATE_POLLUTANTS', 29), ('ECON_GASOLINEPRICE', 32), ('TAX_TERROR_GROUP_HAMAS', 6), ('TAX_FNCACT_MANAGING_DIRECTOR', 88), ('TAX_FNCACT_ADMINISTRATOR', 22), ('TAX_FNCACT_SPOKESPERSON', 185), ('CRISISLEX_C05_NEED_OF_SHELTERS', 28), ('WB_1606_SCHOLARSHIPS', 18), ('TAX_FNCACT_STUDENTS', 173), ('TAX_WORLDMAMMALS_BATS', 41), ('TAX_FNCACT_MILITARY_LEADER', 4), ('EPU_POLICY_REGULATORY', 226), ('STRIKE', 36), ('ECON_MONOPOLY', 32), ('WB_545_NATIONAL_OIL_COMPANIES', 3), ('TAX_WORLDLANGUAGES_TAGALOG', 1), ('TAX_FNCAC

In [14]:
#寻找mention都大于1000次的pair/   Find the pair whose mention are greater than 1000 times
def select(line):
  res = []
  for i,word1 in enumerate(line):
    if word1 in individual_broad.value:
      for j, word2 in enumerate(line):
        if i > j and word2 in individual_broad.value:
          res.append(tuple(sorted([word1, word2])))
  return res


#find frequence pairs
apr_stage2 = Themes_of_CH.flatMap(lambda a:select(a)).map(lambda a:(a, 1)).reduceByKey(lambda a,b:a+b)
dbg(apr_stage2)


[(('EPU_POLICY_CENTRAL_BANK', 'USPEC_POLICY1'), 155), (('TAX_FNCACT_MINISTER', 'WB_2025_INVESTIGATION'), 32), (('TAX_DISEASE', 'TAX_FNCACT_MINISTER'), 192), (('TAX_DISEASE', 'WB_696_PUBLIC_SECTOR_MANAGEMENT'), 595), (('WB_635_PUBLIC_HEALTH', 'WB_831_GOVERNANCE'), 314), (('EPU_ECONOMY_HISTORIC', 'WB_2165_HEALTH_EMERGENCIES'), 647), (('WB_2165_HEALTH_EMERGENCIES', 'WB_831_GOVERNANCE'), 282), (('TAX_FNCACT_MINISTER', 'WB_2166_HEALTH_EMERGENCY_PREPAREDNESS_AND_DISASTER_RESPONSE'), 189), (('WB_2165_HEALTH_EMERGENCIES', 'WB_2166_HEALTH_EMERGENCY_PREPAREDNESS_AND_DISASTER_RESPONSE'), 1263), (('GENERAL_HEALTH', 'UNGP_HEALTHCARE'), 1459), (('CRISISLEX_C04_LOGISTICS_TRANSPORT', 'WB_635_PUBLIC_HEALTH'), 122), (('LEADER', 'WB_832_ANTI_CORRUPTION'), 257), (('GENERAL_HEALTH', 'LEADER'), 829), (('TAX_FNCACT_PRIME_MINISTER', 'WB_696_PUBLIC_SECTOR_MANAGEMENT'), 118), (('TAX_FNCACT_PRIME_MINISTER', 'WB_2025_INVESTIGATION'), 18), (('GENERAL_GOVERNMENT', 'TAX_FNCACT_MINISTER'), 334), (('GENERAL_GOVERNMENT

#   ** function for finding Frequent_Itemsets**



## def Frequent_Itemsets(countryCode):

> input: countryCode for country to Analysis

> ouptut: individual, apr_stage2, n

> ***Return:***
*   individual : item with Number of occurrences in basket
*   apr_stage2 ： item pairs Number of occurrences in basket
*   n ： The whole size of a basket







In [15]:
def select(line):
  res = []
  for i,word1 in enumerate(line):
    if word1 in individual_broad.value:
      for j, word2 in enumerate(line):
        if i > j and word2 in individual_broad.value:
          res.append(tuple(sorted([word1, word2])))
  return res
#put it into function

def Frequent_Itemsets(countryCode):
  """remember to change the frequent_num"""
  frequent_num = 50 #define at least how many occurrences are considered frequent

  Themes_of_Country = Country_And_Theme.filter(lambda a: a[0] == countryCode).map(lambda a: a[1])
  n = Themes_of_Country.count()
  #find frequence set and broadcast
  individual = Themes_of_Country.flatMap(lambda a:[(i,1) for i in a]).reduceByKey(lambda a,b:a+b)
  individual1 = individual.filter(lambda a: a[1] >= frequent_num).collectAsMap()   #change the frequence here !!!!!!!!
  individual_broad = sc.broadcast(individual1)

  #find frequence pairs
  apr_stage2 = Themes_of_Country.flatMap(lambda a:select(a)).map(lambda a:(a, 1)).reduceByKey(lambda a,b:a+b)
  return individual, apr_stage2, n

individual_Pairs_CH, Frequent_Themes_in_CH, count_of_total_pairs= Frequent_Itemsets("CH")
dbg(individual_Pairs_CH)
dbg(Frequent_Themes_in_CH)
dbg(count_of_total_pairs)

[('WB_1235_CENTRAL_BANKS', 178), ('TAX_FNCACT_CEO', 269), ('WB_1045_TREASURY', 129), ('ECON_OILPRICE', 100), ('TAX_ETHNICITY_VENEZUELAN', 6), ('WB_724_HUMAN_RESOURCES_FOR_PUBLIC_SECTOR', 340), ('LEGISLATION', 705), ('WB_2482_RECONCILIATION', 53), ('TAX_WEAPONS_TEAR_GAS', 4), ('WB_1427_NON_COMMUNICABLE_DISEASE_AND_INJURY', 191), ('BULLYING', 30), ('USPEC_UNCERTAINTY1', 529), ('SOC_EMERGINGTECH', 98), ('WB_512_GAS_FIRED_POWER', 5), ('TAX_ETHNICITY_HUNGARIAN', 7), ('WB_1841_SHORT_LIVED_CLIMATE_POLLUTANTS', 29), ('ECON_GASOLINEPRICE', 32), ('TAX_TERROR_GROUP_HAMAS', 6), ('TAX_FNCACT_MANAGING_DIRECTOR', 88), ('TAX_FNCACT_ADMINISTRATOR', 22), ('TAX_FNCACT_SPOKESPERSON', 185), ('CRISISLEX_C05_NEED_OF_SHELTERS', 28), ('WB_1606_SCHOLARSHIPS', 18), ('TAX_FNCACT_STUDENTS', 173), ('TAX_WORLDMAMMALS_BATS', 41), ('TAX_FNCACT_MILITARY_LEADER', 4), ('EPU_POLICY_REGULATORY', 226), ('STRIKE', 36), ('ECON_MONOPOLY', 32), ('WB_545_NATIONAL_OIL_COMPANIES', 3), ('TAX_WORLDLANGUAGES_TAGALOG', 1), ('TAX_FNCAC

# 2. computing the support， confidence and Interest rate


## def support(Frequent_pairs , n):

> Frequent_pairs: item Pairs with Number of occurrences in basket

> total count of Baskets items of the country
  
> ***Return***: support rate 


## def confidence(individual_pairs, Frequent_Themes, count_of_total_pairs):

> individual_pairs: item with Number of occurrences in basket

> Frequent_Themes： item pairs Number of occurrences in basket

> total count of Baskets items of the country

> ***Return***: confidence

## def interest_rate(individual, confidence):

> individual : item with Number of occurrences in basket

> confidence : Pairs with confidence rate

> ***Return***: interest_rate






In [16]:
def support(Frequent_pairs , n):
  """/"""
  Support_For_Pairs = Frequent_pairs.map(lambda a:(a[0], a[1]/count_of_total_pairs))
  return Support_For_Pairs
support_CH = support(Frequent_Themes_in_CH, count_of_total_pairs)

In [17]:
def confidence(individual_pairs, Frequent_Themes, count_of_total_pairs):
  """..."""
  sup_individual = individual_pairs.map(lambda a:(a[0], a[1]/count_of_total_pairs))
  sup_ij = Frequent_Themes.map(lambda a:(a[0][0], (a[0][1], a[1]/count_of_total_pairs)))
  sup_ji = Frequent_Themes.map(lambda a:(a[0][1], (a[0][0], a[1]/count_of_total_pairs)))
  sup_all_ij_and_ji = sup_ij.union(sup_ji)
  all = sup_all_ij_and_ji.join(sup_individual)
  confidence = all.map(lambda a: ((a[0], a[1][0][0]), a[1][0][1]/a[1][1]))
  return confidence

confidence_CH = confidence(individual_Pairs_CH, Frequent_Themes_in_CH, count_of_total_pairs)
dbg(confidence_CH)

[(('WB_1045_TREASURY', 'WB_1281_MANUFACTURING'), 0.14728682170542637), (('WB_1045_TREASURY', 'WB_615_GENDER'), 0.015503875968992248), (('WB_1045_TREASURY', 'WB_838_PUBLIC_ACCOUNTABILITY_MECHANISMS'), 0.007751937984496124), (('WB_1045_TREASURY', 'WB_1467_EDUCATION_FOR_ALL'), 0.1627906976744186), (('WB_1045_TREASURY', 'WB_1484_EDUCATION_SKILLS_DEVELOPMENT_AND_LABOR_MARKET'), 0.15503875968992248), (('WB_1045_TREASURY', 'WB_2467_TERRORISM'), 0.007751937984496124), (('WB_1045_TREASURY', 'WB_670_ICT_SECURITY'), 0.054263565891472874), (('WB_1045_TREASURY', 'WB_939_NEGOTIATION'), 0.015503875968992248), (('WB_1045_TREASURY', 'WB_2465_REVOLUTIONARY_VIOLENCE'), 0.007751937984496124), (('WB_1045_TREASURY', 'WB_2837_IMMIGRATION'), 0.007751937984496124), (('WB_1045_TREASURY', 'WB_346_COMPETITIVE_INDUSTRIES'), 0.14728682170542637), (('WB_1045_TREASURY', 'WB_696_PUBLIC_SECTOR_MANAGEMENT'), 1.0), (('WB_1045_TREASURY', 'WB_831_GOVERNANCE'), 0.10077519379844961), (('WB_1045_TREASURY', 'WB_1668_LABOR_STAN

In [18]:
def interest_rate(individual, confidence):
  interest_join = confidence.map(lambda a:(a[0][1], a)).join(individual)
  interest = interest_join.map(lambda a: (a[1][0][0], a[1][0][1] - a[1][1]))
  return interest

interest_CH  = interest_rate(individual_Pairs_CH, confidence_CH)
dbg(interest_CH)


[(('WB_1045_TREASURY', 'WB_724_HUMAN_RESOURCES_FOR_PUBLIC_SECTOR'), -339.8992248062016), (('EPU_POLICY_REGULATORY', 'WB_724_HUMAN_RESOURCES_FOR_PUBLIC_SECTOR'), -339.8628318584071), (('WB_1235_CENTRAL_BANKS', 'WB_724_HUMAN_RESOURCES_FOR_PUBLIC_SECTOR'), -339.938202247191), (('WB_165_AIR_TRANSPORT', 'WB_724_HUMAN_RESOURCES_FOR_PUBLIC_SECTOR'), -339.89830508474574), (('WB_1777_FORESTS', 'WB_724_HUMAN_RESOURCES_FOR_PUBLIC_SECTOR'), -339.94444444444446), (('WB_2482_RECONCILIATION', 'WB_724_HUMAN_RESOURCES_FOR_PUBLIC_SECTOR'), -339.6792452830189), (('ARMEDCONFLICT', 'WB_724_HUMAN_RESOURCES_FOR_PUBLIC_SECTOR'), -339.894297635605), (('WB_1786_ENVIRONMENTAL_SUSTAINABILITY', 'WB_724_HUMAN_RESOURCES_FOR_PUBLIC_SECTOR'), -339.9672131147541), (('WB_713_PUBLIC_FINANCE', 'WB_724_HUMAN_RESOURCES_FOR_PUBLIC_SECTOR'), -339.8830409356725), (('TAX_FNCACT_PRESIDENT', 'WB_724_HUMAN_RESOURCES_FOR_PUBLIC_SECTOR'), -339.8893249607535), (('TAX_WORLDLANGUAGES_RUSSIA', 'WB_724_HUMAN_RESOURCES_FOR_PUBLIC_SECTOR')

In [19]:
dbg(support_CH.takeOrdered(5, lambda a:-a[1]))
dbg(confidence_CH.takeOrdered(5,lambda a:-a[1]))
dbg(interest_CH.takeOrdered(5,lambda a:a[1]))

[(('TAX_ETHNICITY', 'TAX_FNCACT'), 0.6064264849074976), (('EPU_POLICY', 'TAX_FNCACT'), 0.5464459591041869), (('TAX_FNCACT', 'TAX_WORLDLANGUAGES'), 0.5326192794547225), (('TAX_ETHNICITY', 'TAX_WORLDLANGUAGES'), 0.528919182083739), (('TAX_ETHNICITY', 'TAX_ETHNICITY_CHINESE'), 0.41830574488802336)]
[(('WB_1045_TREASURY', 'WB_696_PUBLIC_SECTOR_MANAGEMENT'), 1.0), (('WB_1045_TREASURY', 'WB_713_PUBLIC_FINANCE'), 1.0), (('EPU_POLICY_REGULATORY', 'EPU_POLICY'), 1.0), (('WB_1235_CENTRAL_BANKS', 'WB_318_FINANCIAL_ARCHITECTURE_AND_BANKING'), 1.0), (('WB_1235_CENTRAL_BANKS', 'WB_1920_FINANCIAL_SECTOR_DEVELOPMENT'), 1.0)]
[(('WB_2372_AUTHENTICATION_AND_AUTHORIZATION', 'TAX_FNCACT'), -4553.590277777777), (('WB_670_ICT_SECURITY', 'TAX_FNCACT'), -4553.308771929825), (('EPU_CATS_REGULATION', 'TAX_FNCACT'), -4553.204887218045), (('WB_1150_VOLATILITY', 'TAX_FNCACT'), -4553.14705882353), (('WB_1174_WAREHOUSING_AND_STORAGE', 'TAX_FNCACT'), -4553.141592920354)]
